 # Opis notatnika

 Ten notatnik inicjuje naszą pracę nad warsztatem końcowym. Naszym zadaniem tutaj jest pobranie udostępnionych nam danych do obszaru roboczego, które w następnym kroku wgramy na naszą bazę danych. Ich obróbka oraz analiza zostanie przeprowadzona w specjalnie do tego celu przygotowanych kolejnych notatnikach.

 Na potrzeby tego warsztatu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.

 Przy wykonywaniu tego zadania możesz posłużyć się tym tokenem: `WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr`.

 ### Uwagi
 - Ze względów ćwiczeniowych, konstrukcja poszczególnych endpointów jest różna – w trakcie pracy dokładnie przyjrzyj się, w jaki sposób należy wykonać zapytanie, aby otrzymać odpowiedź.
 - Pamiętaj o dodaniu `sleep` pomiędzy poszczególnymi wywołaniami endpoint.
 - Limit wywołań API to 500/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

import wymaganych bibliotek

In [1]:
from tqdm import tqdm
import requests
import pandas as pd
import time
import datetime
import sys
sys.path.append('../')


pd.set_option("display.max_colwidth", 3000)
pd.set_option("display.max_columns", None)

definicja parametru połączenia do API

In [2]:
from config.api_datalab import base_url, token

In [3]:
def return_from_api(url, token):
    # print(url)
    try:
        r = requests.get(url, headers={"authorization": token})
    except requests.exceptions.Timeout:
        print('Maybe set up for a retry, or continue in a retry loop')
    except requests.exceptions.TooManyRedirects:
        print('Tell the user their URL was bad and try a different one')
    except requests.exceptions.RequestException as e:
        print('catastrophic error.')
        raise SystemExit(e)
    return r

Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [4]:
airports = pd.read_csv("../data/airports.csv", index_col=None)

In [5]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   origin_airport_id  364 non-null    int64
dtypes: int64(1)
memory usage: 3.0 KB


 # Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.

 ### Wskazówki
 - Nie wszystkie lotniska dostępne w pliku `airports.csv`, są dostępne w endpoint. Zadbaj o odpowiednie obsłużenie takiej sytuacji,
 - Do skonwertowania wyników przydatna może okazać się metoda `Pandas` - [from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html),
 - Artykuł LMS: `Python - analiza danych > Dzień 4 - API > Uwierzytelnianie`
 - Artykuł LMS: `Python - analiza danych > Przygotowanie do zjazdu 2`

 Tutaj pobierz dane z endpoint'u `airport`

In [6]:
def get_airports(airports):
    tmp = []
    wait = 0.2
    all_rows = len(airports)
    td = datetime.timedelta(seconds=all_rows * wait)

    print("Pobieram informację o lotniskach")
    print(f"Wierszy do sprawdzenia: {all_rows}")
    print(f"Czas do zakończenia: {td} [hh:mm:ss]")

    start_time = time.time()
    pbar = tqdm(airports["origin_airport_id"])

    for airport_id in pbar:
        from_api = return_from_api(
            url=f"{base_url}airport/{airport_id}", token=token)

        pbar.set_description(f'Lotnisko {airport_id}')
        all_rows -= 1
        time.sleep(wait)
        if from_api.reason == "OK":
            tmp.append(from_api.json())

        # print(f"\twczytano {len(tmp)} zostało {all_rows}")

    pbar.close()
    end_time = time.time()
    print(
        f'Pobrano wszystkie dane w czasie {datetime.timedelta(seconds=(end_time-start_time))} [hh:mm:ss]')
    return pd.DataFrame(tmp)

In [7]:
airport_df = get_airports(airports=airports)

Pobieram informację o lotniskach
Wierszy do sprawdzenia: 364
Czas do zakończenia: 0:01:12.800000 [hh:mm:ss]


Lotnisko 12511: 100%|██████████| 364/364 [01:38<00:00,  3.70it/s]

Pobrano wszystkie dane w czasie 0:01:38.335432 [hh:mm:ss]


In [8]:
airport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ORIGIN_AIRPORT_ID     97 non-null     int64 
 1   DISPLAY_AIRPORT_NAME  97 non-null     object
 2   ORIGIN_CITY_NAME      97 non-null     object
 3   NAME                  97 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [9]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`.

In [10]:
airport_df.to_csv("../data/raw/airport_list.csv", index=False)

 # Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.

 Wskazówki:
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`, czyli stworzenie listy, a następnie przekonwertowanie jej w postać ramki.
 - Data początkowa danych to `2019-01-01`, zaś data końcowa to `2020-03-31`, czyli 15 miesięcy,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.
 - Przy dodawaniu miesięcy do daty może przydać się metoda [relativedelta](https://www.geeksforgeeks.org/python-get-month-from-year-and-weekday/).

In [11]:
def get_weather_df(start_date, end_date):
    """
    dopisuje wiersze z json do DatataFrames
    """
    tmpDF = pd.DataFrame()
    tmp = 0
    wait = 0.2
    daterange = pd.date_range(start_date, end_date, freq="M").strftime("%Y-%m")
    all_month = len(daterange)
    td = datetime.timedelta(seconds=all_month * wait)

    print("pobieram informacje o pogodzie")
    print(f"Ilość zapytań przez API:{all_month}")
    print(f"Czas do zakończenia powyżej: {td} [hh:mm:ss]")
    start_time = time.time()
    pbar = tqdm(daterange, desc='year_month', unit='month')
    for year_month in pbar:
        from_api = return_from_api(
            url=f"{base_url}airportWeather?date={year_month}", token=token
        )
        all_month -= 1
        time.sleep(wait)
        if from_api.reason == "OK":
            tmp += 1
            tmpDF = pd.concat(
                [tmpDF, pd.DataFrame(from_api.json())], ignore_index=True, axis=0
            )
            # print(f'pobrano dane dla miesiąca {year_month} zostało do pobrania jeszcze {all_month} miesięcy")

    pbar.close()
    end_time = time.time()
    print(
        f'Pobrano wszystkie dane w czasie {datetime.timedelta(seconds=(end_time-start_time))} [hh:mm:ss]')
    return pd.DataFrame(tmpDF)

In [12]:
start_date = "2019-01-01"
end_date = "2020-03-31"

airport_weather_df = get_weather_df(start_date=start_date, end_date=end_date)

/tmp/ipykernel_231242/3693180267.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  daterange = pd.date_range(start_date, end_date, freq="M").strftime("%Y-%m")


pobieram informacje o pogodzie
Ilość zapytań przez API:15
Czas do zakończenia powyżej: 0:00:03 [hh:mm:ss]


year_month: 100%|██████████| 15/15 [00:10<00:00,  1.38month/s]

Pobrano wszystkie dane w czasie 0:00:10.853574 [hh:mm:ss]


In [13]:
airport_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46226 entries, 0 to 46225
Data columns (total 33 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   WT18     0 non-null      object 
 1   STATION  46226 non-null  object 
 2   NAME     46226 non-null  object 
 3   DATE     46226 non-null  object 
 4   AWND     45845 non-null  float64
 5   PRCP     46197 non-null  float64
 6   SNOW     32338 non-null  float64
 7   SNWD     31750 non-null  float64
 8   TAVG     34625 non-null  float64
 9   TMAX     46203 non-null  float64
 10  TMIN     46200 non-null  float64
 11  WDF2     45854 non-null  float64
 12  WDF5     45704 non-null  float64
 13  WSF2     45854 non-null  float64
 14  WSF5     45704 non-null  float64
 15  WT01     16798 non-null  float64
 16  WT08     5589 non-null   float64
 17  WT02     2268 non-null   float64
 18  WT03     5085 non-null   float64
 19  WT04     362 non-null    float64
 20  WT09     316 non-null    float64
 21  WT06     522

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [14]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`.

In [15]:
airport_weather_df.to_csv("../data/raw/airport_weather.csv", index=False)

 # Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft` a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


In [16]:
def get_aircraft():
    tmp = []
    wait = 0.2
    from_api = return_from_api(url=f"{base_url}aircraft", token=token)
    time.sleep(wait)
    print("Pobieram informację o samolotach")
    if from_api.reason == "OK":
        print("Pobrano wszystkie dane")
        return pd.DataFrame(from_api.json())
    print(f"wystąpił błąd dostępu do serwera:{from_api.reason}")
    return from_api.reason

In [17]:
aircraft_df = get_aircraft()

Pobieram informację o samolotach
Pobrano wszystkie dane


In [18]:
aircraft_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7383 entries, 0 to 7382
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MANUFACTURE_YEAR  7383 non-null   int64  
 1   TAIL_NUM          7383 non-null   object 
 2   NUMBER_OF_SEATS   7376 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 173.2+ KB


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [19]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`.


In [20]:
aircraft_df.to_csv("../data/raw/aircraft.csv", index=False)

 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.

In [21]:
def get_flights(airports, start_date, end_date):
    wait = 0.5

    tmpDF = pd.DataFrame()

    daterange = pd.date_range(start_date, end_date, freq="ME").strftime("%Y-%m")
    all_airports = len(airports)
    all_month = len(daterange)
    total_reqest = all_month * all_airports
    td = datetime.timedelta(seconds=total_reqest*wait)

    print(f"Ilość lotnisk: {all_airports}")
    print(f"ilość miesięcy dla każdego z lotnisk: {all_month}")
    print(
        f"Ilość zapytań przez API: {all_month}*{all_airports} = {total_reqest}")
    print(f"Czas do zakończenia powyżej: {td} [hh:mm:ss]")

    pbar = tqdm(airports["origin_airport_id"], unit='airport')
    start_time = time.time()
    for airport_id in pbar:
        for year_month in daterange:
            pbar.set_description(
                f'airport ID {airport_id}, month {year_month}')
            from_api = return_from_api(
                url=f"{base_url}flight?airportId={airport_id}&date={year_month}",
                token=token,
            )
            if from_api.reason == "OK":
                tmpDF = pd.concat(
                    [tmpDF, pd.DataFrame(from_api.json())], ignore_index=True, axis=0
                )
            time.sleep(wait)

    pbar.close()
    end_time = time.time()
    print(
        f'Pobrano wszystkie dane w czasie {datetime.timedelta(seconds=(end_time-start_time))} [hh:mm:ss]')

    return tmpDF

In [22]:
start_date = "2019-01-01"
end_date = "2020-03-31"

flight_df = get_flights(
    airports=airports, start_date=start_date, end_date=end_date)

Ilość lotnisk: 364
ilość miesięcy dla każdego z lotnisk: 15
Ilość zapytań przez API: 15*364 = 5460
Czas do zakończenia powyżej: 0:45:30 [hh:mm:ss]


airport ID 12511, month 2020-03: 100%|██████████| 364/364 [1:58:33<00:00, 19.54s/airport]  

Pobrano wszystkie dane w czasie 1:58:33.332595 [hh:mm:ss]


In [23]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9251880 entries, 0 to 9251879
Data columns (total 27 columns):
 #   Column               Dtype  
---  ------               -----  
 0   MONTH                int64  
 1   DAY_OF_MONTH         int64  
 2   DAY_OF_WEEK          int64  
 3   OP_UNIQUE_CARRIER    object 
 4   TAIL_NUM             object 
 5   OP_CARRIER_FL_NUM    int64  
 6   ORIGIN_AIRPORT_ID    int64  
 7   DEST_AIRPORT_ID      int64  
 8   CRS_DEP_TIME         int64  
 9   DEP_TIME             float64
 10  DEP_DELAY_NEW        float64
 11  DEP_TIME_BLK         object 
 12  CRS_ARR_TIME         int64  
 13  ARR_TIME             float64
 14  ARR_DELAY_NEW        float64
 15  ARR_TIME_BLK         object 
 16  CANCELLED            int64  
 17  CRS_ELAPSED_TIME     float64
 18  ACTUAL_ELAPSED_TIME  float64
 19  DISTANCE             int64  
 20  DISTANCE_GROUP       int64  
 21  YEAR                 int64  
 22  CARRIER_DELAY        float64
 23  WEATHER_DELAY        float64
 24

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [24]:
flight_df_expected_shape = (9251880, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`.

In [25]:
flight_df.to_csv("../data/raw/flight.csv", index=False)

 # Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.

In [26]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego kroku.
